Use RFE to find effective features

In [ ]:
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

# fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

fileList = ['002645.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    y = data['up_down'].iloc[10:].values.tolist()
    X = data.iloc[10:,2:27].copy() # To avoid the case where changing X also changes data, 2 to 27
    X.fillna(0, inplace=True)
    X = scaler.fit_transform(X)
    
#     X, y =  

In [ ]:
estimator = SVR(kernel="linear")
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(X, y)

In [ ]:
selector.support_

In [ ]:
cut_na = selector.ranking_

In [ ]:
cut_na

In [ ]:
fill_na

MINMAXSCALER

In [ ]:
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

# fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

fileList = ['002645.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    y = data['up_down'].iloc[10:].values.tolist()
    X = data.iloc[10:,2:27].copy() # To avoid the case where changing X also changes data, 2 to 27
    X.fillna(0, inplace=True)
    X = scaler.fit_transform(X)

In [ ]:
estimator = SVR(kernel="linear")
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(X, y)

In [ ]:
selector.support_

In [ ]:
selector.ranking_

### RFE test method 1: run RFE for each stock then rank the features

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile

processedList = []
errorList = []
# outfile = TemporaryFile()

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
a_s = np.zeros([1, 49]) # calculated according to X shape
a_r = np.zeros([1, 49]) # calculated according to X shape

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    
    y = data['up_down'].values.tolist()
    X = data.iloc[:,7:56].copy() # To avoid the case where changing X also changes data, 2 to 27
    X.fillna(0, inplace=True)
    
    try:
        X = scaler.fit_transform(X)

        estimator = SVR(kernel="linear")
        selector = RFECV(estimator, step=1, cv=5)
        selector = selector.fit(X, y)
        
        condition = selector.support_

        if len(condition)%49 == 0:
            a_s = np.append(a_s, selector.support_)
            a_r = np.append(a_r, selector.ranking_)
            
            np.save(savePath+'a_s.npy', a_s)
            np.save(savePath+'a_r.npy', a_r)
            
            processedList.append(f)
        else:
            errorList.append(f)
    except:
        errorList.append(f)

# feature ranking array
shape_row = len(processedList) + 1

a_s = a_s.reshape(shape_row, 49)  # calculated according to X shape
a_r = a_r.reshape(shape_row, 49)

a_s = np.delete(a_s, (0), axis=0) # delete first row
a_r = np.delete(a_r, (0), axis=0) # delete first row

In [ ]:
# save errorlist to txt file
with open('errorlist.txt', 'w') as f:
    for item in errorList:
        f.write("%s\n" % item)

In [ ]:
np.sum(a_s, axis=0) #the higher the better

In [ ]:
np.sum(a_r, axis=0) #the lower the better

In [ ]:
data.columns

In [ ]:
selector.score(X,y)

In [ ]:
# plot
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(selector.grid_scores_) + 1), selector.grid_scores_)
plt.show()

### RFE test method 2: preprocess data into one dataframe and run RFE for the large dataframe once and rank 
### Not working very well, too slow for training the model

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile

processedList = []
errorList = []
# outfile = TemporaryFile()

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']


data_new = pd.DataFrame()

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
    data_new = data_new.append(data)
    
    processedList.append(f)
    if len(processedList)%500 == 0:
        fileName = savePath + str(len(processedList)) + '.csv'
        data_new.to_csv(fileName, encoding = 'utf-8', index = False)
        data_new = pd.DataFrame()
        
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    


In [ ]:
# Follow a section of combining all seperated files

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileName = loadPath + '3500combine.csv'
data = pd.read_csv(fileName)

y = data['up_down'].values.tolist()
X = data.iloc[:,7:56].copy() # To avoid the case where changing X also changes data, 2 to 27
X.fillna(0, inplace=True)
    

X = scaler.fit_transform(X)

estimator = SVR(kernel="linear")
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(X, y)

a_s = selector.support_
a_r = selector.ranking_

In [ ]:
l = ['ts_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close']

mid = data[['ts_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close']]
data.drop(labels=['ts_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close'], axis=1,inplace = True)

for i in l:
    data.insert(0, i, mid[i])

### RFE test method 3: move the labels one day ahead, use today indices to predict tomorrow label

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile

processedList = []
errorList = []
# outfile = TemporaryFile()

loadPath = '/Users/jenieshen/Study/graduateThesis/Project/data/processed/Structure/'
savePath = '/Users/jenieshen/Study/graduateThesis/Project/data/processed/normalization/'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
a_s = np.zeros([1, 55]) # calculated according to X shape
a_r = np.zeros([1, 55]) # calculated according to X shape

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    
    y = data['up_down'].values.tolist()
    X = data.iloc[:,1:56].copy() # To avoid the case where changing X also changes data, only excluded the ts code and trading date
    X.fillna(0, inplace=True)
    X = X.values
    
    y0 = np.zeros([1,1])
    y = np.append(y, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y = np.delete(y, (0), axis=0) # delete first row to ensure the lenth is correct
    
    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
    
    try:
        X = scaler.fit_transform(X)

        estimator = SVR(kernel="linear")
        selector = RFECV(estimator, step=1, cv=5)
        selector = selector.fit(X, y)
        
        condition = selector.support_

        if len(condition)%55 == 0:
            a_s = np.vstack((a_s, selector.support_))
            a_r = np.vstack((a_r, selector.ranking_))
            
            np.save(savePath+'a_s_ahead.npy', a_s)
            np.save(savePath+'a_r_ahead.npy', a_r)
            
            processedList.append(f)
        else:
            errorList.append(f)
    except:
        errorList.append(f)

# feature ranking array

a_s = np.delete(a_s, (0), axis=0) # delete first row
a_r = np.delete(a_r, (0), axis=0) # delete first row

In [ ]:
# save errorlist to txt file
with open('errorlist.txt', 'w') as f:
    for item in errorList:
        f.write("%s\n" % item)

In [ ]:
np.sum(a_s, axis=0) #the higher the better

In [ ]:
np.sum(a_r, axis=0) #the lower the better

In [ ]:
data.columns

In [ ]:
idxnameList = data.columns[2:57].tolist()
idxsptList = np.sum(a_s, axis=0).tolist()
idxrkList = np.sum(a_r, axis=0).tolist()

In [ ]:
df_RFE = pd.DataFrame(
    {'Name': idxnameList,
     'Support': idxsptList,
     'Rank': idxrkList
    })

In [ ]:
df_RFE.sort_values('Rank')
temp = df_RFE.sort_values('Rank')['Name'].tolist()
temp[0:15]

In [ ]:
df_RFE.sort_values('Support')
temp = df_RFE.sort_values('Support', ascending = False)['Name'].tolist()
temp[0:15]

In [ ]:
df_RFE.sort_values('Support', ascending = False)

In [ ]:
selector.score(X,y)

In [ ]:
# plot
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(selector.grid_scores_) + 1), selector.grid_scores_)
plt.show()

### RFE test method 4: use today indices to predict the price is higher or lower than one week before, 4th day index predict p_c comparison of 5th day and 1st day

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile

processedList = []
errorList = []
# outfile = TemporaryFile()

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
a_s = np.zeros([1, 56]) # calculated according to X shape
a_r = np.zeros([1, 56]) # calculated according to X shape

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    
    y = data['w_up_down'].values.tolist()
    X = data.iloc[:,2:58].copy() # To avoid the case where changing X also changes data, only excluded the ts code and trading date
    X.fillna(0, inplace=True)
    X = X.values
    
    y0 = np.zeros([1,1])
    y = np.append(y, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y = np.delete(y, (0), axis=0) # delete first row to ensure the lenth is correct
    
    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
    
    try:
        X = scaler.fit_transform(X)

        estimator = SVR(kernel="linear")
        selector = RFECV(estimator, step=1, cv=5)
        selector = selector.fit(X, y)
        
        condition = selector.support_

        if len(condition)%56 == 0:
            a_s = np.vstack((a_s, selector.support_))
            a_r = np.vstack((a_r, selector.ranking_))
            
            np.save(savePath+'a_s_weekly.npy', a_s)
            np.save(savePath+'a_r_weekly.npy', a_r)
            
            processedList.append(f)
        else:
            errorList.append(f)
    except:
        errorList.append(f)

# feature ranking array

a_s = np.delete(a_s, (0), axis=0) # delete first row
a_r = np.delete(a_r, (0), axis=0) # delete first row

In [ ]:
data.columns[2:58]

In [ ]:
idxnameList = data.columns[2:58].tolist()
idxsptList = np.sum(a_s, axis=0).tolist()
idxrkList = np.sum(a_r, axis=0).tolist()

In [ ]:
df_RFE = pd.DataFrame(
    {'Name': idxnameList,
     'Support': idxsptList,
     'Rank': idxrkList
    })

In [ ]:
df_RFE.sort_values('Rank')
temp = df_RFE.sort_values('Rank')['Name'].tolist()
temp[0:15]

In [ ]:
df_RFE.sort_values('Support')
temp = df_RFE.sort_values('Support', ascending = False)['Name'].tolist()
temp[0:15]

In [ ]:
# plot
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(selector.grid_scores_) + 1), selector.grid_scores_)
plt.show()

In [ ]:
df_RFE